In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('./data/cleaned_songs.csv')
df = df.drop(columns=['Unnamed: 0','SName', 'Artist'])
df = df.dropna()
# Test Removing categories
df['Genres'] = df['Genres'].apply(lambda x:  list(set(x.lower().replace("/", ";").replace(" ", "").split(';'))))


In [ ]:
counts = pd.DataFrame(df.Genres.values.tolist()).stack().value_counts()
def remove_small_genres(row, buffer=5000, counts=counts):
    n_list = list()
    for r in row:
        if counts[r]>buffer:
            n_list.append(r)
    if len(n_list) == 0:
        return np.nan
    n_list.sort()
    return n_list
df['Genres'] = df['Genres'].apply(lambda x:  remove_small_genres(x))
df = df.dropna()
counts = pd.DataFrame(df.Genres.values.tolist()).stack().value_counts()
print(len(df))




In [ ]:
# Add columns of labeled values
mlb = MultiLabelBinarizer()
mlb.fit(df.Genres)
Y = mlb.transform(df.Genres)

for i in range(len(mlb.classes_)):
    df.loc[:, [mlb.classes_[i]]] = [y[i] for y in Y]

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])

for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    NB_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = NB_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

In [ ]:
print(NB_pipeline)

In [ ]:
import re
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
ps = PorterStemmer()
stop_words = stopwords.words('english')
test_text = "Out here in the fields, I fight for my mealsI get my back into my living I don't need to fight to prove I'm right I don't need to be forgiven, yeah, yeah, yeah, yeah, yeah Don't cry, don't raise your eye It's only teenage wasteland Sally, take my hand, we'll travel south 'cross land Put out the fire and don't look past my shoulder The exodus is here, the happy ones are near Let's get together before we get much older Teenage wasteland, it's only teenage wasteland Teenage wasteland, oh, yeah Teenage wasteland They're all wasted"

def clean_lyrics(string):
    string = string.lower()
    string.replace('chorus','')
    string = re.sub(r'[^\w]', ' ', string)
    string = " ".join([ps.stem(word) for word in string.split() if (len(word) > 3 and word not in stop_words)])
    return string

test_text = clean_lyrics(test_text)
print(NB_pipeline.predict([test_text]))


In [ ]:
from sklearn.linear_model import LogisticRegression
LogReg_pipeline = Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
        ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    LogReg_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = LogReg_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

In [ ]:
from sklearn.svm import LinearSVC
SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    SVC_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = SVC_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

def clf_creator(gram):
    gram_vectorizer = CountVectorizer(ngram_range=(1, gram))
    gram_vectorizer.fit(df['Lyric'].values)
    vectorizered_gram = gram_vectorizer.transform(df['Lyric'].values)
    tfidf_vectorizer =  TfidfTransformer()
    tfid_vector =  tfidf_vectorizer.fit(vectorizered_gram)
    return Pipeline([('vect', gram_vectorizer),
                            ('tfIdf', tfid_vector),
                          ('clf',BernoulliNB())])


unigram_clf = clf_creator(1)